In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.nn import functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [111]:
torch.cuda.is_available()

True

In [112]:
transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])

trainset = torchvision.datasets.MNIST(root='./data', train=True, 
                                      download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                     download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

### Полносвязная нейронная сеть

In [ ]:
#hyper params
num_epoch = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_d = 28*28
hidden_d = 120
out_d = 10

In [ ]:
class MyModel(nn.Module):
    def __init__(self,
                 input_dim: int,
                 hidden_dim: int,
                 out_dim: int,
                 ):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim, bias=True)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.linear3 = nn.Linear(hidden_dim, out_dim)

        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.linear1(x))
        x = self.activation(self.linear2(x))
        x = self.activation(self.linear3(x))

        return x

In [ ]:
model = MyModel(input_d, hidden_d, out_d)
model = model.to(device)
optim = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
loaders = {"train": trainloader, "valid": testloader}

In [ ]:
accuracy = {"train": [], "valid": []}
for epoch in range(num_epoch):
    epoch_correct = 0
    epoch_all = 0
    for k, dataloader in loaders.items():
        for X_batch, y_batch in tqdm(dataloader):
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            if k == "train":
                model.train()
                optim.zero_grad()
                outp = model(X_batch.view(X_batch.size(0), -1))
                loss = criterion(outp, y_batch)
                loss.backward()
                optim.step()
            else:
                model.eval()
                with torch.no_grad():
                    outp = model(X_batch.view(X_batch.size(0), -1))
            preds = outp.argmax(-1)
            correct = (preds == y_batch).sum()
            all = y_batch.size(0)
            epoch_correct += correct.item()
            epoch_all += all
        if k == "train":
            print(f"Epoch: {epoch+1}")
        print(f"Loader: {k}. Accuracy: {epoch_correct/epoch_all}")
        accuracy[k].append(epoch_correct/epoch_all)

100%|██████████| 15000/15000 [01:23<00:00, 179.33it/s]


Epoch: 1
Loader: train. Accuracy: 0.48396666666666666


100%|██████████| 2500/2500 [00:08<00:00, 290.67it/s]


Loader: valid. Accuracy: 0.4851714285714286


100%|██████████| 15000/15000 [01:21<00:00, 183.59it/s]


...

Loader: valid. Accuracy: 0.6840714285714286


100%|██████████| 15000/15000 [01:18<00:00, 191.96it/s]


Epoch: 10
Loader: train. Accuracy: 0.7369833333333333


100%|██████████| 2500/2500 [00:08<00:00, 304.02it/s]

Loader: valid. Accuracy: 0.7423285714285714


### Сверточная нейронная сеть с архитектурой LeNet + batchnorm

In [143]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.bn1 = nn.BatchNorm2d(6)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.bn2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
        self.bn3 = nn.BatchNorm1d(120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.bn1(F.max_pool2d(F.relu(self.conv1(x)), (2, 2)))
        x = self.bn2(F.max_pool2d(F.relu(self.conv2(x)), 2))
        x = x.view(x.shape[0], -1)
        x = self.bn3(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [147]:
lenet_model = LeNet()
lenet_model = lenet_model.to(device)
optim = torch.optim.Adam(lenet_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [148]:
accuracy = {"train": [], "valid": []}
for epoch in range(num_epoch):
    epoch_correct = 0
    epoch_all = 0
    for k, dataloader in loaders.items():
        for X_batch, y_batch in tqdm(dataloader):
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            if k == "train":
                lenet_model.train()
                optim.zero_grad()
                outp = lenet_model(X_batch)
                loss = criterion(outp, y_batch)
                loss.backward()
                optim.step()
            else:
                lenet_model.eval()
                with torch.no_grad():
                    outp = lenet_model(X_batch)
            preds = outp.argmax(-1)
            correct = (preds == y_batch).sum()
            all = y_batch.size(0)
            epoch_correct += correct.item()
            epoch_all += all
        if k == "train":
            print(f"Epoch: {epoch+1}")
        print(f"Loader: {k}. Accuracy: {epoch_correct/epoch_all}")
        accuracy[k].append(epoch_correct/epoch_all)

100%|██████████| 15000/15000 [02:04<00:00, 120.78it/s]


Epoch: 1
Loader: train. Accuracy: 0.9038833333333334


100%|██████████| 2500/2500 [00:10<00:00, 239.76it/s]


Loader: valid. Accuracy: 0.9138285714285714


100%|██████████| 15000/15000 [02:04<00:00, 120.61it/s]


...

Loader: valid. Accuracy: 0.9732285714285714


100%|██████████| 15000/15000 [02:03<00:00, 121.19it/s]


Epoch: 10
Loader: train. Accuracy: 0.9735166666666667


100%|██████████| 2500/2500 [00:10<00:00, 243.57it/s]

Loader: valid. Accuracy: 0.9755571428571429
